In [ ]:
import pandas as pd
import numpy as np
import sqlalchemy as sa

# API Wrappers

No final da última aula vimos como construir um **wrapper** para uma API: uma classe que nos permite encapsular as etapas de autenticação e acesso à uma API. **Wrappers** nos permitem *abstrair* as camadas de conexão (como nosso script se comunicará com a API) das camadas de dados (o que vamos extrair de informação). Essa separação facilita a construção de códigos mais robustos - tanto à alterações na estrutura da API (simplificando o processo de atualização da camada de conexão) quanto aos requisitos de dados (simplificando o processo de alterações nos dados que extraímos).

Muitas APIs contém **wrappers** pré-implementados através de bibliotecas, nos liberando para focar na extração e manipulação de dados e não na camada de conexão! Hoje veremos uma aplicação desenvolvida a partir do **wrapper** `spotipy` - uma biblioteca que facilita a conexão e extração de dados da plataforma de streaming **Spotify**.

Alguns wrappers interessantes:

* [Dados Abertos Brasil](https://pypi.org/project/DadosAbertosBrasil/)
* [Dados Econômicos World Bank](https://pypi.org/project/wbgapi/)
* [Meteostat para Dados Metereológicos](https://github.com/meteostat/meteostat-python)
* [Yahoo! Finace](https://pypi.org/project/yfinance/)
* [Lista extensiva de Wrappers Python](https://github.com/realpython/list-of-python-api-wrappers) para encontrar outros wrappers interessantes.

Existem inúmeros wrappers pré-construídos - caso você deseje encontrar um wrapper para uma API específica seu melhor amigo será o Google (`python API nome_da_api`)!

## Lidando com autenticação

Como vimos na aula passada, o acesso à muitas APIs é autenticado. É uma boa idéia separar as chaves de autenticação, por exemplo um `token`, do nosso código Python: se tornarmos nosso código público, via GitHub por exemplo, não queremos que outras pessoas utilizem nossas credenciais para acessar uma API!

Podemos utilizar a biblioteca `dotenv` para *esconder* nossas chaves em um arquivo a parte do nosso código:

- Instale a biblioteca usando `!pip install dotenv` (ou `!pip install python-dotenv`)
- Crie um arquivo texto com a extensão `.env` na mesma pasta que o seu código Python
- Insira as as suas chaves de autenticação utilizando a notação abaixo:
```
API_KEY="l1noPOPAixCPM"
API_SECRET="GraGq0zrGhs1qvbA0xQXsZBKuTkK5MJ"
```
- Utilize a função `dotenv.load_dotenv('nome_do_seu_arquivo.env')`
- Agora podemos utilizar a biblioteca `os` para recuperar as variáveis escritas no arquivo utilizando `os.getenv('API_KEY')`

A idéia por trás desse procedimento é que podemos armazenar nossas chaves em um arquivo separado do nosso código. Agora podemos adicionar o arquivo `.env` ao nosso `.gitignore` para que ele não seja sincronizado com nosso repositório!

In [ ]:
!pip install python-dotenv

In [ ]:
import os
from dotenv import load_dotenv

Agora vamos carregar o arquivo `exemplo_aula.env` e verificar que a variável `API_KEY` foi carregada corretamente:

In [ ]:
load_dotenv('credentials/exemplo_aula.env')

In [ ]:
print(os.getenv('API_KEY'))

# Spotipy

Para vermos como wrapper funcionam utilizaremos a biblioteca `spotipy`, que funciona como **wrapper para o Spotify**.

A maior parte dos **wrappers** não fazem parte da instalação padrão do Anaconda, logo precisamos instalá-la utilizando `!pip`.

In [ ]:
!pip install spotipy

Além da própria biblioteca, vamos importar a classe `SpotifyClientCredentials` para fazer a autenticação na API do Spotify.

In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

## Definindo o Problema

Antes de mergulharmos na biblioteca `spotipy`, vamos entender o que queremos fazer com os dados desta API.

Além de fornecer dados sobre a popularidade de artistas, músicas mais tocadas, etc, a API do Spotify fornece dados quantitativos sobre as músicas disponíveis no serviço de streaming. Essas informações são chamadas de *Audio Features*.

Para entender o que são as *Audio Features* devemos ler a (extensa) [documentação](https://developer.spotify.com/documentation/web-api/reference/#/operations/get-several-audio-features) da API.

Nosso objetivo é utilizar essas informações (disponíveis por faixa) para quantificar e diferenciar festivais musicais acontencedo no verão de 2022 na Europa. O primeiro passo é carregar uma tabela com os dados de *headliners* para cada festival:

In [ ]:
import pandas as pd

def read_from_gsheets(spreadsheet_link):
    """
    Transform Google Sheets URL into a CSV file
    """
    working_spreadsheet = spreadsheet_link.replace(
        "/edit?usp=sharing", "/export?format=csv"
    )

    return pd.read_csv(working_spreadsheet)


In [ ]:
tb_festivals = read_from_gsheets(
    "https://docs.google.com/spreadsheets/d/1aUiwstZKEENiw3KAT1CCCXcji1iLWdCL_HwngvLxzJc/edit?usp=sharing"
)
tb_festivals.head(10)


## Conectando ao Spotify

Para nos conectarmos com a API do Spotify precisamos criar um par de `CLIENT_ID` e `CLIENT_SECRET`. Podemos fazer isso através do dashboard de desenvolvedor do Spotify: https://developer.spotify.com.

Vamos guardar o par de chaves criado em um arquivo `.env` e carrega-lo utilizando a função `load_dotenv`:

In [ ]:
load_dotenv('credentials/spotify.env')

Agora vamos utilizar a classe `SpotifyClientCredentials` para criar o gerenciador de autenticação da API. Utilizando este gerenciador, podemos inicializar a conexão com a API utilizando a classe `spotipy.Spotify`:

In [ ]:
auth_manager = SpotifyClientCredentials(
    client_id=os.getenv('CLIENT_ID'), 
    client_secret=os.getenv('CLIENT_SECRET')
)
spotify = spotipy.Spotify(client_credentials_manager=auth_manager)

### Investigando Audio Features

Vamos investigar o que são os *audio features* de algumas músicas para entender como extrair essa informação.

Inicialmente, precisamos do *url* de cada faixa no Spotify:

In [ ]:
dict_songs = {
    "kate_bush": "https://open.spotify.com/track/75FEaRjZTKLhTrFGsfMUXR?si=26398bff72014b5a",
    "slayer": "https://open.spotify.com/track/4fiOTntQKr24p07FvQDHZE?si=148a03ca6ba844fb",
    "nin": "https://open.spotify.com/track/27tX58NOpv1YKQ0abW7EPy?si=09484ca5e58d454a",
    "cardi_b": "https://open.spotify.com/track/58q2HKrzhC3ozto2nDdN4z?si=6f755b3e29d841ad",
}


Agora vamos utilizar o método `audio_features` para extrair, a partir de um URL de faixa do Spotify, os dados de uma música específica:

In [ ]:
spotify.audio_features(dict_songs["cardi_b"])

## Conectando **Festivais** à **Audio Features**

Agora que vimos como extrair *audio features* a partir de um URI de faixa, vamos mapear o caminho que utilizaremos para conectar nossa tabela de *headliners* aos *audio features*.

Precisamos *sair* de uma tabela de **nomes de artista** para o **URI de faixa** para algumas faixas de cada artista. O primeiro passo então é buscar informações sobre o artista a partir do nome deste!

### Extraindo Artistas

Para buscar *strings* na API do Spotify podemos utilizar o método `.search()`. Vamos utilizar este método para buscar um artista específico e investigar os resultados.

In [ ]:
search_result = spotify.search(q="Cardi B", type="artist")
print(search_result)

Um dicionário bem complexo... Podemos tentar tratar esse dicionário utilizando a função `json_normalize()` da biblioteca Pandas:

In [ ]:
pd.json_normalize(search_result)

Obviamente o resultado não foi o esperado... Precisamos investigar mais diretamente o resultado do método:

In [ ]:
search_result.keys()

In [ ]:
search_result

Com o URI em mãos, podemos utilizar o método `.artist()` para trazer apenas um artista específico:

In [ ]:
dict_cardib = search_result['artists']['items'][0]
dict_cardib

In [ ]:
dict_cardib['uri']

In [ ]:
spotify.artist(dict_cardib['uri'])

### Extraindo Faixas

Agora precisamos transformar o URI de artista em um conjunto de faixas. Como extrair todas as músicas de um dado artista pode demorar muito, vamos utilizar o método `.artist_top_tracks()` para extrair dados das 10 maiores músicas de cada artista:

In [ ]:
top_10_tracks = spotify.artist_top_tracks(dict_cardib['uri'])
top_10_tracks

Outro dicionário complexo... Novamente, tentemos trata-lo utilizando `json_normalize()`

In [ ]:
pd.json_normalize(top_10_tracks)

Novamente, não é o resultado esperado...

In [ ]:
top_10_tracks.keys()

In [ ]:
type(top_10_tracks['tracks'])

In [ ]:
top_10_tracks['tracks'][0].keys()

In [ ]:
[track['name'] for track in top_10_tracks['tracks']]

In [ ]:
[track['uri'] for track in top_10_tracks['tracks']]

In [ ]:
top10_cardib_uri = [track['uri'] for track in top_10_tracks['tracks']]

### Extraindo Audio Features

Agora, com o URI de cada uma das 10 maiores músicas de um artista, podemos utilizar o método `.audio_features()` para extrair os *audio features* de cada uma dessas músicas:

In [ ]:
spotify.audio_features(top10_cardib_uri[2])

## Construindo nosso DB

Para construir um DB que possa responder nossa pergunta original, precisamos:

1. Consolidar os métodos que analisamos nas etapas anteriores;
1. Determinar quais **entidades** nosso banco representará;
1. Escrever o código para extrair as informações necessárias e carrega-la no DB.

Podemos estruturar nosso DB ao longo de 3 tabelas:

1. **`headliner`**: tabela com a relação entre festivais e artistas;
1. **`artist`**: tabela com informação de artistas;
1. **`track`**: tabela com Audio Features (e outras informações de faixa).

Antes de mais nada vamos nos conectar ao nosso DB:

In [ ]:
load_dotenv('credentials/mysql.env')
url_banco = "localhost"
nome_db = "spotify"
conn_str = f"mysql+pymysql://{os.getenv('MYSQL_USER')}:{os.getenv('MYSQL_PASS')}@{url_banco}/{nome_db}"
engine = sa.create_engine(conn_str)

### Tabela `headliner`

A tabela `headliner` deve conter as informações de quais artistas vão tocar em quais festivais. Poderíamos utilizar a tabela que carregamos do Google Sheets diretamente, mas isso faria o cruzamento entre **headliner** e **artist** pelo nome do artista (que pode ser frágil).

Vamos utilizar a API do Spotify para enriquecer a nossa tabela original com os URIs de cada artista:

In [ ]:
tb_festivals.head()

In [ ]:
array_headliner = tb_festivals['headliners'].unique()
array_headliner[0:10]

In [ ]:
dict_uri = dict()
for artista in array_headliner:
    search_result = spotify.search(q=artista, type="artist")
    dict_uri[artista] = search_result['artists']['items'][0]['uri']

In [ ]:
dict_uri = dict()
for artista in array_headliner:
    try:
        search_result = spotify.search(q=artista, type="artist")
        dict_uri[artista] = search_result['artists']['items'][0]['uri']
    except IndexError as e:
        dict_uri[artista] = np.nan

In [ ]:
dict_uri

In [ ]:
tb_festivals['artist_uri'] = tb_festivals['headliners'].map(dict_uri)
tb_festivals.head()

In [ ]:
tb_festivals[tb_festivals['artist_uri'].isna()]

In [ ]:
tb_festivals.dropna().to_sql('headliner', engine, index = False, if_exists = 'replace')

### Tabela `artist`

A tabela `artist` deve conter informações dos artistas presentes na tabela `headliner`. Vamos começar selecionando apenas os URIs distintos da tabela `headliner`:

In [ ]:
artist_uri = engine.execute('SELECT DISTINCT artist_uri FROM headliner').fetchall()
artist_uri

In [ ]:
dados_artista = []
for artista in artist_uri:
    search_result = spotify.artist(artista[0])
    dados_artista.append(
        (
            search_result['uri'],
            search_result['name'],
            search_result['popularity'],
            search_result['followers']['total']
        )
    )
dados_artista[0:5]

In [ ]:
tb_artista = pd.DataFrame(dados_artista, columns = ['artist_uri', 'artist_name', 'popularity', 'followers'])
tb_artista.head()

In [ ]:
tb_artista.to_sql('artist', engine, index = False, if_exists = 'replace')

### Tabela `tracks`

Agora, com os URIs de cada artista, podemos utilizar os métodos `.artist_top_tracks()` e `.audio_features()` para extrair as informações de cada faixa (tanto informações gerais, como o nome, quanto os *audio features* em si):

In [ ]:
top_10_tracks = spotify.artist_top_tracks('spotify:artist:0C0XlULifJtAgn6ZNCW2eu')
top_10_tracks

In [ ]:
[track['uri'] for track in top_10_tracks['tracks']]

In [ ]:
top_10_tracks['tracks'][0].keys()

In [ ]:
top_10_tracks['tracks'][0]['name']

In [ ]:
spotify.audio_features(top_10_tracks['tracks'][0]['uri'])

In [ ]:
track_data = []
for artist in artist_uri:
    top_tracks = spotify.artist_top_tracks(artist[0])
    for track in top_tracks['tracks']:
        track_au = spotify.audio_features(track['uri'])[0]
        track_au['name'] = track['name']
        track_au['popularity'] = track['popularity']
        track_au['explicit'] = track['explicit']
        track_au['artist_uri'] = artist[0]
        track_data.append(track_au)

tb_tracks = pd.DataFrame(track_data)
tb_tracks.head()

In [ ]:
tb_tracks.to_sql('track', engine, index = False, if_exists = 'replace')

## Usando nosso DB

In [ ]:
query_festival = '''
    SELECT
        h.festival,
        AVG(t.valence) AS feliz,
        AVG(t.energy) AS energia,
        AVG(t.danceability) AS dancavel,
        AVG(a.popularity) AS pop,
        AVG(a.followers) AS seguidores
    FROM 
        headliner h JOIN
        artist a ON (h.artist_uri = a.artist_uri) JOIN
        track t ON (a.artist_uri = t.artist_uri)
    GROUP BY
        h.festival 
'''
tb_festival_au = pd.read_sql(query_festival, engine)
tb_festival_au.head()

In [ ]:
tb_festival_au.sort_values('seguidores', ascending=False).head()